In [1]:
import torch
#https://zorba-blog.tistory.com/22
#$ pip install torchmetrics
torch.cuda.is_available() == True

True

In [2]:
torch.cuda.is_available()


True

In [3]:
torch.cuda.get_device_name

<function torch.cuda.get_device_name(device: Union[torch.device, str, int, NoneType] = None) -> str>

In [4]:
torch.cuda.device_count()

1

In [12]:
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
# ClassificationDeployArguments

In [13]:
args = ClassificationTrainArguments(
    #미리 훈련된 모델 bert-base-uncased 구글에서 공개한 다국어 모데류
    pretrained_model_name='beomi/kcbert-base',
    downstream_corpus_name='nsmc',
    downstream_model_dir='checkpoint-doccls',
    batch_size = 32,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=1,
    tpu_cores=0,
    seed=7,
    downstream_corpus_root_dir='korpora/'
)

In [14]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='korpora/', downstream_model_dir='checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)


set seed: 7


In [15]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir = args.downstream_corpus_root_dir,
    force_download=True
)

print(args.downstream_corpus_root_dir)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 33.3MB/s]                                                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 15.9MB/s]                                                             

korpora/


In [16]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False
)

In [17]:
tokenizer

BertTokenizer(name_or_path='beomi/kcbert-base', vocab_size=30000, model_max_length=300, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [18]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args = args,
    corpus=corpus,
    tokenizer = tokenizer,
    mode='train'
)

train_dataset

INFO:ratsnlp:Loading features from cached file korpora/nsmc\cached_train_BertTokenizer_128_nsmc_document-classification [took 6.214 s]


In [19]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 2170, 832, 5045, 17, 17, 7992, 29734, 4040, 10720, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [30]:
from torch.utils.data import DataLoader, RandomSampler
# train_dataloader = DataLoader(
#     train_dataset,
#     batch_size=args.batch_size,
#     sampler=RandomSampler(train_dataset, replacement=False),
#     collate_fn=nlpbook.data_collator,
#     drop_last=False,
#     num_workers=args.cpu_workers
# )

train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

train_dataloader

In [21]:
from torch.utils.data import SequentialSampler
val_dataset = ClassificationDataset(
    args=args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode='test'
)

val_dataset

INFO:ratsnlp:Loading features from cached file korpora/nsmc\cached_test_BertTokenizer_128_nsmc_document-classification [took 2.300 s]


In [22]:
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers
)

In [23]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [24]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model,args)

In [25]:
trainer=nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

C:\ProgramData\Anaconda3\envs\39\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:608: UserWarning: Checkpoint directory D:\developer\test\test\bert_gpt\checkpoint-doccls exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\ProgramData\Anaconda3\envs\39\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\ProgramData\Anaconda3\envs\39\lib\site-packages\pytorch_lightning\core\optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 

Training: 0it [00:00, ?it/s]

In [32]:
#trainer.fit(task, train_dataloader=train_dataloader, val_dataloaders=val_dataloader)

# trainer.fit(task, train_dataloader=train_dataloader, val_dataloaders=val_dataloader)